# LLMRouterChain

In [2]:
import openai
import os 
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base= os.getenv('OPENAI_API_BASE')
openai.api_type= "azure"
openai.api_version = os.getenv('OPENAI_API_VERSION')
deployment:str=os.getenv('CHATGPT_MODEL')

llm = AzureChatOpenAI(
    deployment_name=deployment,
    model_name=deployment
    )


NameError: name 'AzureChatOpenAI' is not defined

In [ ]:
os.environ['OPENAI_API_KEY'] = ''


In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')


In [ ]:
llm = ChatOpenAI()


In [ ]:
hr_template="""You are a very smart human resources person. \
You are great at answering questions about human resources in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


In [ ]:
law_template="""You are a very smart lawyer. \
You are great at answering questions about the law in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


In [ ]:
prompt_infos = [
{
    'name':'human resources',
    'description':'Answer human resources questions',
    'template':hr_template},

{
    'name':'law',
    'description':'Answers legal questions',
    'template':law_template}
]


In [ ]:
destination_chains ={}


In [ ]:
for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm,prompt=prompt)
    destination_chains[name] = chain


In [ ]:
default_prompt = ChatPromptTemplate.from_template('{input}')

default_chain = LLMChain(llm=llm,prompt=default_prompt)


In [ ]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)


In [ ]:

destinations=[
f"{p['name']}: {p['description']}" for p in prompt_infos
]


In [ ]:
destinations


In [ ]:
destination_str = "\n".join(destinations)

print(destination_str)


In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

router_template


In [ ]:
router_prompt = PromptTemplate(template=router_template,
input_variables=['input'],
output_parser=RouterOutputParser())

router_chain = LLMRouterChain.from_llm(llm,router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
destination_chains=destination_chains,
default_chain=default_chain,verbose=True)

chain.run("How do intellectual property laws address the use of AI-generated content?")


In [ ]:
chain.run("How do you handle conflict between team members?")
